In [1]:
!pip3 install --upgrade pip setuptools wheel
!pip3 install torch numpy
!pip3 install git+https://github.com/openai/whisper.git 

  Cloning https://github.com/openai/whisper.git to /private/var/folders/wc/zvrqyr9n1hv9nw1xjnbts6l00000gn/T/pip-req-build-4iw8im9q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/wc/zvrqyr9n1hv9nw1xjnbts6l00000gn/T/pip-req-build-4iw8im9q
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!brew install portaudio
!pip3 install pyaudio
!pip3 install SpeechRecognition

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
falcosecurity-libs  gcr                 libgudev            yalantinglibs

You have 6 outdated formulae installed.

To reinstall 19.7.0, run:
  brew reinstall portaudio


Speech to text

In [3]:
import whisper
import pyaudio
import wave
import numpy as np
import time

# Load Whisper model
model = whisper.load_model("base")  # You can choose a different model like 'small', 'medium', 'large' etc.

# Setup microphone input
p = pyaudio.PyAudio()

# Define recording parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 25  # Record for 15 seconds

# Create a list to store transcribed text
transcribed_text = []

# Function to record and transcribe
def record_and_transcribe():
    print("Recording...")

    # Open the audio stream
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK)

    frames = []

    # Record for 15 seconds
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")
    stream.stop_stream()
    stream.close()

    # Save the recorded audio to a temporary file
    wf = wave.open("temp.wav", 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    # Transcribe audio using Whisper
    result = model.transcribe("temp.wav")
    transcription = result["text"]

    # Append the transcription to the list
    transcribed_text.append(transcription)
    print(f"Transcription: {transcription}")

    # Optionally, clean up the temp file
    # os.remove("temp.wav")

# Main loop to record and transcribe
record_and_transcribe()

# Display the final transcription list
print("\nFinal Transcription:")
print(transcribed_text)


Recording...
Recording finished.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  Hi my name is Sona Selvraj, I am currently pursuing my Masters in Computer Science at Clark University. I am looking for data on others and machine learning engineering roles.

Final Transcription:
[' Hi my name is Sona Selvraj, I am currently pursuing my Masters in Computer Science at Clark University. I am looking for data on others and machine learning engineering roles.']


In [4]:
!pip3 install pyttsx3
!pip3 install gtts

Text to Speech

In [5]:
import pyttsx3

def text_to_speech(text):
    engine = pyttsx3.init()  # Initialize the TTS engine
    
    # Set properties (optional)
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1.0)  # Volume level (0.0 to 1.0)

    # Speak the given text
    engine.say(text)
    engine.runAndWait()

if __name__ == "__main__":
    # text = input("Enter text to convert to speech: ")
    text_to_speech("thank you. Your response is recorded")



In [6]:
import os
import platform
from gtts import gTTS

def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")

    # Detect OS and play the audio
    if platform.system() == "Windows":
        os.system("start output.mp3")  # Windows
    elif platform.system() == "Darwin":  # macOS
        os.system("afplay output.mp3")
    elif platform.system() == "Linux":
        os.system("mpg321 output.mp3")  # Install mpg321 using: sudo apt install mpg321

if __name__ == "__main__":
    # text = input("Enter text to÷ convert to speech: ")
    text_to_speech("thank you. Your response is recorded")


In [16]:
!ollama --version


ollama version is 0.5.13


In [17]:
!ollama list


NAME             ID              SIZE      MODIFIED     
llama2:latest    78e26419b446    3.8 GB    24 hours ago    


In [18]:
!ollama run llama2 "Can you take a mock interview for me for data analyst role?"


⠙ ⠹ ⠹ ⠼ ⠴ ⠴ ⠧ ⠇ Of course! I'd be happy to help you prepare for your interview. Could you p
 provide some details about the position you are applying for, such a
 the company, job description, and any specific skills or experiences the
 looking for in a candidate? This will help me tailor the mock inte
 to your needs.



In [10]:
!ollama run llama2 "If I provide response to the above questions can you provide feedback?"

⠙ 
Of course! I'd be happy to provide feedback on your responses. Please go ah
 answer the questions, and I'll do my best to offer constructive f
5l.



In [11]:
import subprocess

def chat_with_llama():
    print("Welcome to Llama Chat! Type 'exit' to quit.\n")
    
    chat_history = []  # Store chat history
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        
        # Format the conversation history
        chat_history.append(f"User: {user_input}")
        context = "\n".join(chat_history)  # Combine past messages

        # Send full conversation history to Llama
        result = subprocess.run(["ollama", "run", "llama2", context], capture_output=True, text=True)

        response = result.stdout.strip()
        print("\nLlama:", response)

        # Save model's response in history
        chat_history.append(f"Llama: {response}")

chat_with_llama()


Welcome to Llama Chat! Type 'exit' to quit.




You:  exit


Goodbye!


In [1]:
import subprocess
import time

# Store answers
answers = []

def ask_role_and_generate_questions():
    print("Please enter the role you are interviewing for:")
    role = input("Role: ")

    # Refined prompt to only ask for 3 relevant questions without extra text or explanations
    prompt = f"Generate three mock interview questions for a {role} role. Provide only the questions with no extra text or explanations. The questions should be relevant, one technical, one behavioral, and one situational."

    # Get the questions from Llama based on the role
    result = subprocess.run(
        ["ollama", "run", "llama2", prompt], capture_output=True, text=True
    )
    
    # Clean up the output to get only the questions
    questions = result.stdout.strip().split("\n")
    
    # Only keep the actual questions, removing unwanted prefixes
    questions = [q for q in questions if q and not any(prefix in q.lower() for prefix in ['technical', 'behavioral', 'situational'])]

    if len(questions) < 3:
        print("Something went wrong. Please try again.")
        return
    
    print(f"\n--- Questions for the role of {role} ---")
    for i, question in enumerate(questions):
        print(f"Question {i+1}: {question}")
    
    print("\nStarting the interview. Answer the following questions one by one.\n")
    
    # Ask each question and collect answers
    for i, question in enumerate(questions):
        print(f"Question {i+1}: {question}")
        answer = input("Your answer: ")
        answers.append(answer)  # Store the answer
        print("\n" + "-"*50 + "\n")
        time.sleep(1)  # Give some time before the next question
    
    # Provide dynamic feedback
    provide_dynamic_feedback(questions)

def provide_dynamic_feedback(questions):
    print("\n--- Feedback ---")
    
    for i, answer in enumerate(answers):
        # Send the answer to Llama to generate feedback dynamically
        prompt = f"Provide concise feedback for this answer based on best practices for interview responses.\nAnswer: {answer}\nFeedback:"
        
        # Call Llama to get feedback dynamically
        result = subprocess.run(
            ["ollama", "run", "llama2", prompt], capture_output=True, text=True
        )
        
        feedback = result.stdout.strip()
        
        print(f"\nQuestion {i+1}: {questions[i]}")
        print(f"Your answer: {answer}")
        print(f"Feedback: {feedback}")
        print("\n" + "-"*50 + "\n")

# Start the process
ask_role_and_generate_questions()


Please enter the role you are interviewing for:


Role:  java developer



--- Questions for the role of java developer ---
Question 1: What is your experience with multi-threading in Java, and how have you used it to improve application performance?
Question 2: Can you describe a time when you had to work with a difficult team member or stakeholder? How did you handle the situation, and what was the outcome?
Question 3: Suppose you are working on a critical project with a tight deadline, and one of your team members is unable to complete their tasks on time. How would you handle this situation, and how would you ensure that the project is completed on time and to a high standard?

Starting the interview. Answer the following questions one by one.

Question 1: What is your experience with multi-threading in Java, and how have you used it to improve application performance?


Your answer:  worked on client data using multi-threading



--------------------------------------------------

Question 2: Can you describe a time when you had to work with a difficult team member or stakeholder? How did you handle the situation, and what was the outcome?


Your answer:  during my capstone project



--------------------------------------------------

Question 3: Suppose you are working on a critical project with a tight deadline, and one of your team members is unable to complete their tasks on time. How would you handle this situation, and how would you ensure that the project is completed on time and to a high standard?


Your answer:  ask more time or do it by myself



--------------------------------------------------


--- Feedback ---

Question 1: What is your experience with multi-threading in Java, and how have you used it to improve application performance?
Your answer: worked on client data using multi-threading
Feedback: Great, thank you for sharing your experience with multi-threading! Here are some specific areas where you could provide more detail or context to make your answer even stronger:

1. Clarify the purpose of the project: Could you tell me a little bit more about the project you worked on? What was the main goal, and how did using multi-threading help you achieve it?
2. Provide specific examples: Instead of just saying "worked on client data," could you give an example or two of what specifically you did with multi-threading? For instance, did you use a particular threading algorithm or library? How did that help optimize the performance of the application?
3. Explain the benefits: Why did you choose to use multi-threading in th

In [4]:
import whisper
import pyttsx3  # For text-to-speech
import speech_recognition as sr  # For speech-to-text
import numpy as np
import wave
import time

# Initialize Whisper for STT
whisper_model = whisper.load_model("base")  # Or use another model size
recognizer = sr.Recognizer()

# Initialize pyttsx3 for TTS
engine = pyttsx3.init()

# Function to generate speech
def text_to_speech(text):
    engine.say(text)
    engine.runAndWait()

# Function for speech-to-text using Whisper
def speech_to_text():
    with sr.Microphone() as source:
        print("Listening for your answer...")
        audio = recognizer.listen(source)
        print("Processing your answer...")

        # Convert AudioData to WAV file
        with open("user_audio.wav", "wb") as f:
            f.write(audio.get_wav_data())

        # Load the audio file with Whisper
        result = whisper_model.transcribe("user_audio.wav")
        return result['text']

# Function to simulate model interaction (Generate a question)
def generate_question(role):
    if role == "java developer":
        return "What is your experience with multithreading in Java?"
    elif role == "data analyst":
        return "Can you explain how you would approach cleaning a large dataset?"
    else:
        return "Tell me about a time when you worked on a team project."

# Main interaction loop
def interview(role):
    # Generate the question
    question = generate_question(role)
    
    # Convert the question to speech
    print(f"Model: {question}")
    text_to_speech(question)
    
    # Get user's response through speech-to-text
    user_response = speech_to_text()
    
    # Print and return the user's response
    print(f"User's Response: {user_response}")
    
    # Add a mechanism to break out of the loop after the user gives a valid answer
    if len(user_response.strip()) > 0:
        return user_response
    else:
        print("No valid input detected, please try again.")
        return None

# Start the interview
role = input("Please enter the role you are interviewing for: ")

while True:
    response = interview(role)
    if response:
        break  # Exit the loop once a valid response is given
    else:
        time.sleep(2)  # Wait for a bit before retrying


Please enter the role you are interviewing for:  jav developer


Model: Tell me about a time when you worked on a team project.
Listening for your answer...
Processing your answer...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


User's Response:  I have worked on a team project doing my capstone in my file.


In [5]:
##Integrated

In [6]:
import whisper
import pyaudio
import wave
import numpy as np
import time
import os
import platform
import subprocess
from gtts import gTTS

# Initialize Whisper model
model = whisper.load_model("base")  # You can choose a different model like 'small', 'medium', 'large' etc.

# Setup microphone input
p = pyaudio.PyAudio()

# Define recording parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 10  # Adjust the recording duration

# Initialize list to store answers
answers = []

# Function for speech-to-text using Whisper
def record_and_transcribe():
    print("Recording...")

    # Open the audio stream
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK)

    frames = []

    # Record for the defined duration
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")
    stream.stop_stream()
    stream.close()

    # Save the recorded audio to a temporary file
    wf = wave.open("temp.wav", 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    # Transcribe the audio using Whisper
    result = model.transcribe("temp.wav")
    transcription = result["text"]

    # Return the transcription
    return transcription

# Function for text-to-speech using gTTS
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")

    # Play the audio file based on the OS
    if platform.system() == "Windows":
        os.system("start output.mp3")  # Windows
    elif platform.system() == "Darwin":  # macOS
        os.system("afplay output.mp3")
    elif platform.system() == "Linux":
        os.system("mpg321 output.mp3")  # Make sure mpg321 is installed on Linux

# Function to generate interview questions based on the role
def ask_role_and_generate_questions():
    print("Please enter the role you are interviewing for:")
    role = input("Role: ")

    prompt = f"Generate three mock interview questions for a {role} role. Provide only the questions with no extra text or explanations. The questions should be relevant, one technical, one behavioral, and one situational."

    # Generate questions using the Llama model
    result = subprocess.run(
        ["ollama", "run", "llama2", prompt], capture_output=True, text=True
    )

    # Clean and format the output to get only the questions
    questions = result.stdout.strip().split("\n")
    questions = [q.strip() for q in questions if q.strip()]

    if len(questions) < 3:
        print("Something went wrong. Please try again.")
        return

    # Print the questions
    print(f"\n--- Questions for the role of {role} ---")
    for i, question in enumerate(questions):
        print(f"Question {i+1}: {question}")

    # Proceed with the interview
    print("\nStarting the interview. Answer the following questions one by one.\n")

    # Ask each question and collect answers
    for i, question in enumerate(questions):
        # Convert question to speech
        text_to_speech(question)

        # Capture and transcribe the user's answer
        print(f"Listening for your answer...")
        answer = record_and_transcribe()
        
        # Store the answer
        answers.append(answer)
        print(f"Your answer: {answer}")
        print("\n" + "-" * 50 + "\n")
        time.sleep(1)  # Give some time before the next question

    # Provide dynamic feedback based on answers
    provide_dynamic_feedback(questions)

# Function to provide dynamic feedback using the model
def provide_dynamic_feedback(questions):
    print("\n--- Feedback ---")
    
    for i, answer in enumerate(answers):
        # Send the answer to Llama to generate feedback dynamically
        prompt = f"Provide concise feedback for this answer based on best practices for interview responses.\nAnswer: {answer}\nFeedback:"
        
        # Get feedback from the Llama model
        result = subprocess.run(
            ["ollama", "run", "llama2", prompt], capture_output=True, text=True
        )
        
        feedback = result.stdout.strip()
        
        print(f"\nQuestion {i+1}: {questions[i]}")
        print(f"Your answer: {answer}")
        print(f"Feedback: {feedback}")
        print("\n" + "-" * 50 + "\n")

# Start the process
ask_role_and_generate_questions()


Please enter the role you are interviewing for:


Role:  java developer



--- Questions for the role of java developer ---
Question 1: 1. How do you optimize the performance of a Java application by using multi-threading?
Question 2: 2. Can you describe a time when you had to work on a project with a tight deadline? How did you manage your time and resources to meet the deadline?
Question 3: 3. Imagine you are working on a team project, and one of your colleagues is not pulling their weight. How would you handle the situation in a professional manner?

Starting the interview. Answer the following questions one by one.

Listening for your answer...
Recording...
Recording finished.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Your answer:  Ah, Kak Summan aku bana. Pudila, ah, okey aku mulai ya. Namah.

--------------------------------------------------

Listening for your answer...
Recording...
Recording finished.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Your answer:  bhajda

--------------------------------------------------

Listening for your answer...
Recording...
Recording finished.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Your answer: コリエナーコリエナーコリエナーエナズイエンゲワンサッタンゲークだ

--------------------------------------------------


--- Feedback ---

Question 1: 1. How do you optimize the performance of a Java application by using multi-threading?
Your answer:  Ah, Kak Summan aku bana. Pudila, ah, okey aku mulai ya. Namah.
Feedback: While the candidate's response is creative and shows personality, it may not be the most effective answer in an interview setting for several reasons:

1. Lack of clarity: The candidate's response is difficult to understand due to the use of slang and colloquial language. It would be better to use clear and concise language that can be easily understood by the interviewer.
2. Lack of focus: The candidate's response does not directly address the question being asked. It would be better to provide a clear and focused answer that addresses the interviewer's concerns.
3. Inappropriate tone: The candidate's response comes across as casual and disrespectful, which may not be appropriate in a p